In [2]:
%load_ext autoreload
%autoreload 2
%pwd

'/home/luca/uni/master/dl-in-vc/voxl3r'

In [3]:
# setup PYTHONPATH
import sys
sys.path += ['.', './extern/scannetpp', './extern/mast3r', './extern/mast3r/dust3r']

In [4]:
from pathlib import Path
import torch
from torchvision.io import read_image

import pyvista as pv
pv.set_jupyter_backend('client')

from dataset import SceneDataset, SceneDatasetTransformToTorch
from einops import rearrange
from models.surface_net_baseline.model import SimpleOccNetConfig
from models.surface_net_baseline.module import LRConfig, OccSurfaceNet, OptimizerConfig

from utils.data_parsing import load_yaml_munch
from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
import lightning as pl

from utils.visualize import visualize_mesh
from utils.basic import get_default_device


In [5]:
import numpy as np
max_seq_len = 4
dataset = SceneDataset(
        data_dir="./datasets/scannetpp/data",
        camera="iphone",
    )

TypeError: SceneDataset.__init__() got an unexpected keyword argument 'data_dir'

In [5]:
# scene dataset output
data = dataset[0]
mesh = data["mesh"]
path_images = data["path_images"]
camera_params = data["camera_params"]

In [6]:
from utils.chunking import create_chunk

# get chunk of mesh based of camera
data_chunk = create_chunk(mesh.copy(), list(camera_params.keys())[10], camera_params, max_seq_len=8, image_path=path_images, with_backtransform=True)
mesh_chunk = data_chunk["mesh"]
image_names_chunk = data_chunk["image_names"]
camera_params_chunk = data_chunk["camera_params"]
p_center = data_chunk["p_center"]

In [7]:
visualize_mesh(mesh_chunk, images=image_names_chunk, camera_params_list=camera_params_chunk.values())

Widget(value='<iframe src="http://localhost:65081/index.html?ui=P_0x2aa4e12e0_0&reconnect=auto" class="pyvista…

In [1]:

from utils.chunking import mesh_2_voxels

# voxelize mesh and store in training dict
voxel_grid, coordinates, occupancy_values = mesh_2_voxels(mesh_chunk, voxel_size=0.02)

trainings_dict = {
    "mesh": mesh_chunk,
    "training_data" : (coordinates, occupancy_values),
    "images" : (image_names_chunk, camera_params_chunk.values())
}

ModuleNotFoundError: No module named 'common'

In [12]:
visualize_mesh(voxel_grid.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)

Widget(value='<iframe src="http://localhost:65081/index.html?ui=P_0x2aafa6b10_4&reconnect=auto" class="pyvista…

In [13]:
#visualize_mesh(voxel_grid.as_boxes(), point_coords=coordinates[occupancy_values == 1], images=image_names_chunk, camera_params_list=camera_params_chunk.values())

In [14]:
from models.surface_net_baseline.train import visualize_unprojection

visualize_unprojection(trainings_dict)

/Users/luis/uni/dl-vc/voxl3r/dataset.py:112: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  points = torch.tensor(torch.from_numpy(points).float()).to(self.target_device)
/Users/luis/uni/dl-vc/voxl3r/dataset.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt = torch.tensor(torch.from_numpy(gt).float()).to(self.target_device)
/Users/luis/uni/dl-vc/voxl3r/.venv/lib/python3.12/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Widget(value='<iframe src="http://localhost:65081/index.html?ui=P_0x2aaefad80_5&reconnect=auto" class="pyvista…

In [ ]:
from models.surface_net_baseline.train import visualize_unprojection_whole_scene


visualize_unprojection_whole_scene(dataset)

## Scene Chunking

In [ ]:
from dataset import plot_occupency_grid


coordinates, occupancy_values = dataset.create_voxel_grid(idx)
plot_occupency_grid({
  "training_data" : (coordinates, occupancy_values),
  "mesh" : dataset.data_dir / dataset.scenes[idx] / "scans" / "mesh_aligned_0.05.ply",
})

In [ ]:
import trimesh

mesh = trimesh.load(dataset.data_dir / dataset.scenes[idx] / "scans" / "mesh_aligned_0.05.ply")
voxel_grid = mesh.voxelized(0.02)
boxes = voxel_grid.as_boxes()
visualize_mesh(mesh.voxelized(0.02).as_boxes(), opacity=1)

In [ ]:
visualize_mesh(
        dataset.data_dir / dataset.scenes[idx] / "scans" / "mesh_aligned_0.05.ply"

    )

In [ ]:
image_names, camera_params_list, camera_center = data["images"]

visualize_mesh(
        data["mesh"],
        images=image_names,
        camera_params_list=camera_params_list,
        point_coords=camera_center,
    )

## Occlupancy Computation

## Visualization: Transform to Voxel Grid

In [ ]:
from dataset import plot_occupency_grid

plot_occupency_grid(data, resolution=dataset.resolution / 1.1)

## Project RGB Values to Scene

In [ ]:
from models.surface_net_baseline.train import visualize_unprojection

visualize_unprojection(data)

## First Training Run Naive Approach

In [ ]:
from models.surface_net_baseline.data import OccSurfaceNetDatamodule


datamodule = OccSurfaceNetDatamodule(
        dataset,
        ["0cf2e9402d", "0cf2e9402d", "0cf2e9402d"],
        batch_size=1,
        max_sequence_length=max_seq_len,
        single_chunk=False,
    )

In [ ]:
# no pos enc w8mnl00u
# pos first 02th0eov
from utils.visualize import plot_voxel_grid


dict = torch.load(
        "./.lightning/occ-surface-net/surface-net-baseline/lpacvi7v/best_ckpts.pt"
    )
model_path = dict["best_model_val_accuracy"]

# trainer.test(model_path, dataloaders=datamodule)
datamodule.prepare_data()
datamodule.setup("test")

model = OccSurfaceNet.load_from_checkpoint(model_path)
test_dict = model.test_visualize(datamodule.test_dataloader())

gt = torch.cat(test_dict["gt"])
points = torch.cat(test_dict["points"])
y = torch.sigmoid(torch.cat(test_dict["out"]))
y[y < 0.5] = 0.0
y[y > 0.5] = 1.0

mesh = (
    Path(dataset.data_dir)
    / dataset.scenes[datamodule.scene_idx_test]
    / "scans"
    / "mesh_aligned_0.05.ply"
)
mesh = None
# visualize_mesh(mesh, point_coords=points.cpu().numpy(), heat_values=y.cpu().numpy())
plot_voxel_grid(
    points.detach().cpu().numpy(),
    y.detach().cpu().numpy(),
    ref_mesh=mesh,
    resolution=dataset.resolution,
    )


In [ ]:
from models.surface_net_baseline.data import OccSurfaceNetDatamodule
from utils.visualize import plot_voxel_grid

max_seq_len = 10
scene_dataset = SceneDataset(data_dir="datasets/scannetpp/data", camera="iphone", n_points=300000, threshold_occ=0.01, representation="occ", visualize=True, max_seq_len=max_seq_len)
scene_dataset.seed = 42
model_path = ".lightning/occ-surface-net/surface-net-baseline/r3cf6z5t/checkpoints/epoch=262-step=24459.00-val_accuracy=0.89.ckpt"

model = OccSurfaceNet.load_from_checkpoint(model_path)
datamodule = OccSurfaceNetDatamodule(scene_dataset, "8b2c0938d6", batch_size=128, max_sequence_length=max_seq_len)
datamodule.setup('test')
test_dict = model.test_visualize(datamodule.test_dataloader())

gt = torch.cat(test_dict['gt'])
points = torch.cat(test_dict['points'])
y = torch.sigmoid(torch.cat(test_dict['out']))
#y[y < 0.5] = 0.0
#y[y > 0.5] = 1.0

mesh = Path(scene_dataset.data_dir) / scene_dataset.scenes[datamodule.scene_idx] / "scans" / "mesh_aligned_0.05.ply"
visualize_mesh(mesh, point_coords=points.detach().cpu().numpy(), heat_values=y.detach().cpu().numpy())
#plot_voxel_grid(points.detach().cpu().numpy(), y.detach().cpu().numpy(), ref_mesh=mesh)

In [ ]:
occ = y.clone()
occ[y < 0.5] = 0.0
occ[y > 0.5] = 1.0
plot_voxel_grid(points.detach().cpu().numpy(), occ.detach().cpu().numpy(), ref_mesh=mesh)